In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import datasets, transforms

# use GPU for computation if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# download and extract the MNIST data set
trainset = datasets.MNIST(root='./data', train=True, download=True,
                          transform=transforms.ToTensor())
testset = datasets.MNIST(root='./data', train=False, download=True,
                         transform=transforms.ToTensor())

# extract a complete PyTorch dataset
def extract(dataset):
    datasize = len(dataset)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=datasize, shuffle=False)
    return next(iter(dataloader))

# extract all training and test images and labels into PyTorch tensors
train_images, train_labels = extract(trainset)
test_images, test_labels = extract(testset)

# flatten training and test images to vectors with 28 x 28 = 784 entries
# and move to the GPU if available
train_images = train_images.view(-1, 784).to(device)
test_images = test_images.view(-1, 784).to(device)

# Variational Autoencoder

In this notebook we will train and analyze a variational autoencoder for the MNIST data set. Our goal is to perform dimensionality reduction: we reduce the high-dimensional images to a two-dimensional representation that still captures some of the important aspects of the images. We will analyze how well images can be reconstructed from the lower dimensional representations and try to generate images that look similar to the images in the MNIST data set.

## Before you start

It is strongly recommended that you complete the first part of the computer lab and perform a probabilistic PCA on the MNIST data before working through this notebook. You can download the notebook for the PPCA model [here](https://uu-sml.github.io/course-apml-public/lab/PPCA.ipynb) and run it on your computer, or you can [open it on Google Colab](https://colab.research.google.com/github/uu-sml/course-apml-public/blob/master/lab/PPCA.ipynb).

In this notebook we use [PyTorch](https://pytorch.org/), an open source software library for machine learning. Make sure that you have installed the latest version of PyTorch if you run the notebook on your computer. Alternatively, you can [open it on Google Colab](https://colab.research.google.com/github/uu-sml/course-apml-public/blob/master/lab/PPCA.ipynb), which also allows you to use GPUs which might speed up some computations.

A Jupyter notebook with an introduction to PyTorch can be downloaded from [here](https://uu-sml.github.io/course-sml-public/lab/introduction.ipynb). Alternatively, you can [open it on Google Colab](https://colab.research.google.com/github/uu-sml/course-sml-public/blob/master/lab/introduction.ipynb). Reading and running the notebook is highly recommended, since it introduces important concepts and commands that are required in this notebook.

## Motivation

Let us reflect on the limitations of PCA and PPCA. In both models, data is encoded in a lower dimensional space by a linear (or rather affine) transformation, and the lower dimensional representation can be decoded by another affine function. Is it reasonable to assume that we can generate images similar to the ones in the MNIST data set by a linear transformation of low dimensional latent variables?

One implication is that relative distances are preserved between the original space and the lower-dimensional latent space: if two data samples are "close" to each other, then also their latent encodings are relatively "close" to each other, and similarly two latent encodings that are "close" to each other are decoded to two relatively "close" data points. This affects our ability to reconstruct images from their encodings and to generate new images.

### Task

Go back to the PCA (exercise 8.1) and PPCA models. Did you observe this behaviour, i.e., were encodings of digits that look similar close to each other?

## A nonlinear model

Let us consider a more flexible nonlinear model that is given by 
\begin{align*}
  p_{\boldsymbol{\theta}}(\mathbf{x} \,|\, \mathbf{z}) &= \mathcal{N}\left(\mathbf{x}; \mu_{\boldsymbol{\theta}}(\mathbf{z}), \sigma_{\boldsymbol{\theta}}^2 \mathbf{I}_{784}\right), \\
  p_{\boldsymbol{\theta}}(\mathbf{z}) &= \mathcal{N}(\mathbf{z}; \boldsymbol{0}, \mathbf{I}_2),
\end{align*}
where parameters $\boldsymbol{\theta}$ include variance parameter $\sigma^2_{\boldsymbol{\theta}} > 0$ and $\mu_{\boldsymbol{\theta}} \colon \mathbb{R}^2 \to \mathbb{R}^{784}$ is a nonlinear model of the mean of the decoding distribution. In this notebook we model $\mu_{\boldsymbol{\theta}}$ by a neural network and include its weights and biases in $\boldsymbol{\theta}$, but other models could be used equally well.

This nonlinear model is very similar to the probabilistic PCA model - the only difference is that the mean of the decoding distribution $p_{\boldsymbol{\theta}}(\mathbf{x} \,|\, \mathbf{z})$ is not a linear function of $\mathbf{z}$ (namely $\mathbf{W}z + \mu$) but a nonlinear function $\mu_{\boldsymbol{\theta}}(\mathbf{z})$ of $\mathbf{z}$.

A major implication of the nonlinearity of $\mu_{\boldsymbol{\theta}}$, and hence a major difference from the probabilistic PCA model, is: **The marginal distribution $p_{\boldsymbol{\theta}}(\mathbf{x})$ is not a normal distribution, and typically there exists no closed-form expression for it.**

Below you can see an implementation of a specific form of this nonlinear model with PyTorch. The function $\mu_{\boldsymbol{\theta}}$ is modeled by a shallow neural network (its weights and biases are included in $\boldsymbol{\theta}$). The `decode` function outputs the representative decoding $\mu_{\boldsymbol{\theta}}(\mathbf{z})$ for a batch of encodings $\mathbf{z}$.

In [ ]:
class NonLinearModel(nn.Module):

    def __init__(self):
        super(NonLinearModel, self).__init__()

        # linear parts of the nonlinear function mu_theta
        self.decoder_fc1 = nn.Linear(2, 400)
        self.decoder_fc2 = nn.Linear(400, 784)

        # logarithm of variance sigma_theta^2
        self.logsigma2 = nn.Parameter(torch.zeros(1))

    def decode(self, z):
        # z is a matrix in which each row is a two-dimensional encoding
        h1 = F.relu(self.decoder_fc1(z))
        return self.decoder_fc2(h1)

### Task

Read through the definition of the nonlinear model. How is $\mu_{\boldsymbol{\theta}}(\mathbf{z})$, the mean of of the decoding distribution $p_{\boldsymbol{\theta}}(\mathbf{x} | \mathbf{z})$, defined in this model?

## Training

Remember that we trained the probabilistic PCA model by minimizing the negative log-likelihood $\sum_{i=1}^N \log p_{\boldsymbol{\theta}}(\mathbf{x}_i)$ with gradient descent. However, since there is no closed-form expression of $p_{\boldsymbol{\theta}}(\mathbf{x})$ for the nonlinear model, unfortunately we can not implement and optimize the negative log-likelihood in the same straightforward way here.

Instead we optimize an approximation of the negative log-likelihood that we are able to compute: We approximate $p_{\boldsymbol{\theta}}(\mathbf{x})$ with the Monte Carlo estimate
\begin{equation}\label{eq:MC}
  p_{\boldsymbol{\theta}}(\mathbf{x}) = \int p_{\boldsymbol{\theta}}(\mathbf{x} \,|\, \mathbf{z}) p_{\boldsymbol{\theta}}(\mathbf{z}) \,\mathrm{d}\mathbf{z} \approx \frac{1}{K} \sum_{n=1}^K p_{\boldsymbol{\theta}}(\mathbf{x} \,|\, \mathbf{z}_n),
\end{equation}
where $\mathbf{z}_1, \ldots, \mathbf{z}_K$ are i.i.d. samples of $\mathbf{z}$ drawn from the prior $p_{\boldsymbol{\theta}}(\mathbf{z})$. Thus we obtain
\begin{equation*}
  \begin{split}
  \log p_{\boldsymbol{\theta}}(\mathbf{x}) &\approx \log{\left(\sum_{n=1}^K \mathcal{N}(\mathbf{x}; \mu_{\boldsymbol{\theta}}(\mathbf{z}_n), \sigma_{\boldsymbol{\theta}}^2 \mathbf{I}_{784}) \right)} - \log K \\
  &= - 392 (\log \sigma_{\boldsymbol{\theta}}^2 + \log{(2\pi)}) + \log{\left(\sum_{n=1}^K \exp{\left(-\frac{1}{2\sigma_{\boldsymbol{\theta}}^2} {\|\mathbf{x} - \mu_{\boldsymbol{\theta}}(\mathbf{z}_n)\|}_2^2 \right)}\right)} - \log K.
  \end{split}
\end{equation*}
Hence similar to the probabilistic PCA model, given a training data set $\mathbf{x}_1, \ldots, \mathbf{x}_N$ we will optimize the parameters $\boldsymbol{\theta}$ by minimizing the cost function
\begin{equation*}
  J(\boldsymbol{\theta}) = 392 \log \sigma_{\boldsymbol{\theta}}^2 - \frac{1}{N} \sum_{n'=1}^N \log{\left(\sum_{n=1}^K \exp{\left(-\frac{1}{2\sigma_{\boldsymbol{\theta}}^2} {\|\mathbf{x}_{n'} - \mu_{\boldsymbol{\theta}}(\mathbf{z}_n)\|}_2^2 \right)}\right)}
\end{equation*}
with gradient descent.

### Task

Complete the following implementation `cost_function(X, model, K)` of the cost function $J(\boldsymbol{\theta})$ that takes the data matrix $\mathbf{X}$ (each row corresponds to one image), the nonlinear `model`, and the number $K$ of samples $\mathbf{z}_1, \ldots, \mathbf{z}_K$ as inputs. In the code, the sampling of $\mathbf{z}_1, \ldots, \mathbf{z}_K$ and the computation of the decodings $\mu_{\boldsymbol{\theta}}(\mathbf{z}_1), \ldots, \mu_{\boldsymbol{\theta}}(\mathbf{z}_K)$ is missing.

*Hint*: You can sample a PyTorch matrix of size $n \times m$ with standard normally distributed entries with [`torch.randn(n, m)`](https://pytorch.org/docs/stable/torch.html#torch.randn).

In [ ]:
def cost_function(X, model, K):
    # sample z_1, ..., z_K
    # WRITE YOUR CODE HERE

    # compute matrix mu of size K x 784 whose rows are mu(z_1), ..., mu(z_K)
    # WRITE YOUR CODE HERE: mu = ...

    # compute y with y[j, i] = - 1/(2 * sigma_theta^2) * |x_j - mu_theta(z_i))|^2_2
    y = - 0.5 * torch.exp(-model.logsigma2) * (X.view(-1, 1, 784) - mu.view(1, -1, 784)).pow(2).sum(dim=2)
    
    # compute loss
    return 392 * model.logsigma2 - y.logsumexp(dim=1).mean()


We train the nonlinear model with stochastic gradient descent using random batches of 500 images of the MNIST training data set.

In [ ]:
# define data loaders
# `pin_memory=True` is helpful when working with GPUs: https://pytorch.org/docs/stable/data.html#memory-pinning
train_data = torch.utils.data.DataLoader(
    trainset, batch_size=500, shuffle=True, pin_memory=device.type=='cuda'
)
test_data = torch.utils.data.DataLoader(
    testset, batch_size=500, pin_memory=device.type=='cuda'
)

We train the model for 20 epochs. In every iteration we use $K=5$ randomly sampled latent vectors $\mathbf{z}_1, \ldots, \mathbf{z}_K$ to estimate the cost function.

In [ ]:
# define the model and move it to the GPU if available
model = NonLinearModel()
model = model.to(device)

# define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

# use K = 5 samples z_1, ..., z_K
K = 5

# track the training and test loss
training_loss = []
test_loss = []

# optimize parameters for 20 epochs
for i in range(20):
    # for each minibatch
    for batch_X, _ in train_data:
        # move batch to the GPU if available
        batch_X = batch_X.to(device)

        # reset the gradient information
        optimizer.zero_grad()

        # evaluate the cost function on the training data set
        loss = cost_function(batch_X, model, K)

        # update the statistics
        training_loss.append(loss.item())
        test_loss.append(float('nan'))

        # perform backpropagation
        loss.backward()

        # perform a gradient descent step
        optimizer.step()

    # evaluate the model after every epoch
    with torch.no_grad():
        # evaluate the cost function on the test data set
        accumulated_loss = 0
        for batch_X, _ in test_data:
            # move batch to the GPU if available
            batch_X = batch_X.to(device)

            # compute loss
            loss = cost_function(batch_X, model, K)
            accumulated_loss += loss.item()

        # update the statistics
        test_loss[-1] = accumulated_loss / len(test_data)
            
    print(f"Epoch {i + 1:2d}: training loss {training_loss[-1]: 9.3f}, "
          f"test loss {test_loss[-1]: 9.3f}")
        
# plot the tracked statistics
plt.figure()
iterations = np.arange(1, len(training_loss) + 1)
plt.scatter(iterations, training_loss, label='training loss')
plt.scatter(iterations, test_loss, label='test loss')
plt.legend()
plt.xlabel('iteration')
plt.show()

### Task

Read through and try to understand the implementation of the training procedure above. Answer the following questions:

- How does it differ from the implementation of the training procedure of the probabilistic PCA model?
- How were the parameters initialized?
- What learning rate did we use in the gradient descent algorithm?
- For how many iterations was the model trained?

## Generating images

We can use the trained model to generate new images, in the same way as we did with the PPCA model. Again we sample 25 encodings $\mathbf{z}_1, \ldots, \mathbf{z}_{25}$ from $\mathcal{N}(\boldsymbol{0}, \mathbf{I}_{2})$, and plot their representative decodings $\mu_{\boldsymbol{\theta}}(\mathbf{z}_n)$.

In [ ]:
# plot a grid of `images` with `nrows` rows and `ncols` columns
def plot_images(images, nrows=5, ncols=5):
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(2 * ncols, 2 * nrows))
    for image, ax in zip(images, axes.flat):
        # configure subplot
        ax.set_xticks(())
        ax.set_yticks(())
        ax.grid(False)
        
        # plot image
        ax.imshow(image.view(28, 28).cpu().numpy(),
                  vmin=0.0, vmax=1.0, cmap='gray_r')
        
    return fig

### Task

When we generated images from the probabilistic PCA model, we noted that sometimes the quality of samples can be improved by tuning the temperature of the prior. Repeat the sampling with different temperatures and study how it affects the quality of the samples.

*Hint:* Check the notebook with the probabilistic PCA model for the definition of sampling with temperature.

## Variational autoencoder

There is a major problem with the nonlinear model: Not only do we not know the exact form of $p_{\boldsymbol{\theta}}(\mathbf{x})$ but due to the nonlinearity of $\mu_{\boldsymbol{\theta}}(\mathbf{z})$ **we also do not know the encoding distribution** $p_{\boldsymbol{\theta}}(\mathbf{z} \,|\, \mathbf{x})$. Therefore we can not encode images in the lower dimensional latent space!

Variational autoencoders (VAEs) solve this problem by extending the nonlinear model with an encoding distribution $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$ that may depend on $\mathbf{x}$ and some parameters $\boldsymbol{\phi}$. The distribution $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$ is used as an approximation of $p_{\boldsymbol{\theta}}(\mathbf{z} \,|\, \mathbf{x})$. If $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$ is a good approximation of $p_{\boldsymbol{\theta}}(\mathbf{z} \,|\, \mathbf{x})$ we can use it to compute low-dimensional representations of the data in the latent space.

Here, we study a VAE of the form
\begin{align*}
  p_{\boldsymbol{\theta}}(\mathbf{x} \,|\, \mathbf{z}) &= \mathcal{N}\left(\mathbf{x}; \mu_{\boldsymbol{\theta}}(\mathbf{z}), \sigma_{\boldsymbol{\theta}}^2 \mathbf{I}_{784}\right), \\
  p_{\boldsymbol{\theta}}(\mathbf{z}) &= \mathcal{N}(\mathbf{z}; \boldsymbol{0}, \mathbf{I}_2), \\
  q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x}) &= \mathcal{N}\Big(\mathbf{z}; \mu_{\boldsymbol{\phi}}(\mathbf{x}), \operatorname{diag}\big(\sigma_{\boldsymbol{\phi}}^2(\mathbf{x})\big)\Big),
\end{align*}
where, as for the nonlinear model above, parameters $\boldsymbol{\theta}$ include variance parameter $\sigma^2_{\boldsymbol{\theta}} > 0$ and $\mu_{\boldsymbol{\theta}} \colon \mathbb{R}^2 \to \mathbb{R}^{784}$ is a nonlinear model of the mean of the decoding distribution, and the encoding distribution $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$ is a normal distribution with mean $\mu_{\boldsymbol{\phi}}(\mathbf{x}) \in \mathbb{R}^2$ and diagonal covariance matrix with diagonal elements $\sigma_{\boldsymbol{\phi}}^2(\mathbf{x})$.

The following code block defines a specific VAE. The mean and the logarithm of the diagonal entries of the covariance matrix of the encoding distributions for a batch of inputs are returned by `encode`. As above, the `decode` function outputs the mean of the decoding distributions for a batch of encodings.

In [ ]:
class VAE(nn.Module):

    def __init__(self):
        super(VAE, self).__init__()

        # linear parts of the nonlinear encoder
        self.encoder_fc1 = nn.Linear(784, 400)
        self.encoder_fc2_mean = nn.Linear(400, 2)
        self.encoder_fc2_logsigma2 = nn.Linear(400, 2)

        # linear parts of the nonlinear function mu_phi
        self.decoder_fc1 = nn.Linear(2, 400)
        self.decoder_fc2 = nn.Linear(400, 784)

        # logarithm of variance sigma_phi^2
        self.logsigma2 = nn.Parameter(torch.zeros(1))

    def encode(self, X):
        """
        Return mean and log diagonal of the covariance matrix of the
        encoding distribution.
        """
        # flatten the data into a matrix with 28 x 28 = 784 columns
        X = X.view(-1, 784)
        
        h1 = F.relu(self.encoder_fc1(X))
        return self.encoder_fc2_mean(h1), self.encoder_fc2_logsigma2(h1)

    def decode(self, Z):
        """
        Return mean of the decoding distribution.
        """
        h1 = F.relu(self.decoder_fc1(Z))
        return self.decoder_fc2(h1)

### Task

Read through and try to understand the definition of the variational autoencoder. How is $\mu_{\boldsymbol{\theta}}(\mathbf{z})$, the mean of the decoding distribution $p_{\boldsymbol{\theta}}(\mathbf{x} \,|\, \mathbf{z})$ defined? How are $\mu_{\boldsymbol{\phi}}(\mathbf{x})$ and $\sigma^2_{\boldsymbol{\phi}}(\mathbf{x})$, the mean and the diagonal of the covariance matrix of the encoding distribution $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$, defined?

## Training

When we train the VAE we have to optimize both the parameters $\boldsymbol{\theta}$ of the nonlinear model and the parameters $\boldsymbol{\phi}$ of the encoding distribution $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$. 
If we manage that distribution $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$ becomes equal to the distribution $p_{\boldsymbol{\theta}}(\mathbf{z} \,|\, \mathbf{x})$ (or at least close to it), we have found a way to encode our data: for a given data sample $\mathbf{x}$, we can just sample the latent encoding from $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$.

### Theoretical background

For all $\mathbf{x}$ we have
\begin{equation*}
    \log p_{\boldsymbol{\theta}}(\mathbf{x}) =
    \mathbb{E}_{\mathbf{z} \sim q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})}\big[\log p_{\boldsymbol{\theta}}(\mathbf{x} \,|\, \mathbf{z})\big]
    + \mathrm{KL}\big(q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x}) \,\big\|\, p_{\boldsymbol{\theta}}(\mathbf{z} \,|\,\mathbf{x})\big) - \mathrm{KL}\big(q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x}) \,\big\|\, p_{\boldsymbol{\theta}}(\mathbf{z})\big),
\end{equation*}
(we ignore here that the KL-divergence $\mathrm{KL}(p \,\|\, q)$ is only defined if $q(x) = 0$ implies $p(x) = 0$).

The KL divergence of two distributions is always non-negative, and zero if and only if the two distributions are equal. Hence we have for all $\mathbf{x}$
\begin{equation*}
  \log p_{\boldsymbol{\theta}}(\mathbf{x}) \geq
  \underbrace{\mathbb{E}_{\mathbf{z} \sim q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})}\big[\log p_{\boldsymbol{\theta}}(\mathbf{x} \,|\, \mathbf{z})\big]
  - \mathrm{KL}\big(q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x}) \,\big\|\, p_{\boldsymbol{\theta}}(\mathbf{z})\big)}_{\text{evidence lower bound (ELBO)}}
\end{equation*}
with equality if and only if $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$ is equal to
the distribution $p_{\boldsymbol{\theta}}(\mathbf{z} \,|\, \mathbf{x})$. Since the
right-hand side of this inequality is a lower bound of the (log) evidence $\log p_{\boldsymbol{\theta}}(\mathbf{x})$, it is called **evidence lower bound (ELBO)**.

The ELBO can not be evaluated exactly but it can be estimated:
1. The first term $\mathbb{E}_{\mathbf{z} \sim q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})}\big[\log p_{\boldsymbol{\theta}}(\mathbf{x} \,|\, \mathbf{z})\big]$ can be estimated with the Monte Carlo estimate
   \begin{equation*}
       \frac{1}{K} \sum_{i=1}^K \log p_{\boldsymbol{\theta}}(\mathbf{x} \,|\, \mathbf{z}_i)
   \end{equation*}
   where $\mathbf{z}_1,\ldots,\mathbf{z}_K$ are i.i.d. samples from $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$.
2. From the preparatory exercises we know that for our choices of $p_{\boldsymbol{\theta}}(\mathbf{z})$ and $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$ the second term is
   \begin{equation*}
       \mathrm{KL}\big(q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x}) \,\big\|\, p_{\boldsymbol{\theta}}(\mathbf{z})\big) = \frac{1}{2} \left(\sum_{n=1}^2 (\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}))_n + \|\mu_{\boldsymbol{\phi}}(\mathbf{x})\|^2_2 - 2 - \sum_{n=1}^2 \log (\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}))_n\right).
   \end{equation*}
   Since the form of the prior $p_{\boldsymbol{\theta}}(\mathbf{z})$ and $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$ are design choices, usually we can choose them such that there is a closed-form expression for the second term of the ELBO.

What is the advantage of the ELBO over the evidence $\log p_{\boldsymbol{\theta}}(\mathbf{x})$ if one has to use a Monte Carlo estimate as well?

The fundamental difference is that in the estimation of the ELBO the Monte Carlo samples $\mathbf{z}_1, \ldots, \mathbf{z}_K$ are sampled from $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$ instead of $p_{\boldsymbol{\theta}}(\mathbf{z}) = \mathcal{N}(\mathbf{z}; \boldsymbol{0}, \mathbf{I}_2)$: We can tune the parameters $\boldsymbol{\phi}$ such that the terms $p_{\boldsymbol{\theta}}(\mathbf{x} \,|\, \mathbf{z}_i)$ in the Monte Carlo estimate increase (intuitively, it becomes more likely to reconstruct $\mathbf{x}$ from the samples $\mathbf{z}_i$, and hence more terms of the Monte Carlo estimate actually contribute to the estimate.

This observation motivates the idea of maximizing the ELBO by training parameters $\boldsymbol{\theta}$ and $\boldsymbol{\phi}$ simultaneously instead of maximizing the evidence $\log p_{\boldsymbol{\theta}}(\mathbf{x})$ directly by training only $\boldsymbol{\theta}$. If the encoding distribution is flexible enough, we might even be able to obtain $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x}) = p_{\boldsymbol{\theta}}(\mathbf{z} \,|\, \mathbf{x})$, in which case the ELBO is equal to the evidence.

### Cost function

Let our training data $\mathbf{x}_1, \ldots, \mathbf{x}_N$ be i.i.d. samples of $\mathbf{x}$. Then the
joint ELBO, i.e., the sum of the ELBOs for $\mathbf{x}_1, \ldots, \mathbf{x}_N$ separately, is a lower
bound of
\begin{equation*}
    \log p_{\boldsymbol{\theta}}(\mathbf{x}_1, \ldots, \mathbf{x}_N) = \sum_{n=1}^N \log p_{\boldsymbol{\theta}}(\mathbf{x}_n).
\end{equation*}
For our choices of $p_{\boldsymbol{\theta}}(\mathbf{z})$ and $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$ the joint ELBO can be estimated by
\begin{equation*}
    \begin{split}
    & - \frac{1}{2}\left(784 N \left(\log{(2\pi)} + \log{\sigma_{\boldsymbol{\theta}}^2})\right) + \frac{1}{\sigma_{\boldsymbol{\theta}}^2}\sum_{n=1}^N \frac{1}{K} \sum_{n'=1}^K \|\mu_{\boldsymbol{\theta}}(\mathbf{z}_{n,n'}) - \mathbf{x}_n\|^2_2 \right) \\
    & - \frac{1}{2} \sum_{n=1}^N \sum_{i=1}^2 \left( (\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}_n))_i + (\mu_{\boldsymbol{\phi}}(\mathbf{x}_n))^2_i - 1 - \log (\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}_n))_i\right),
     \end{split}
\end{equation*}
where $\mathbf{z}_{n,1}, \ldots, \mathbf{z}_{n,K}$ are independent samples of the encoding distribution $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x}) = \mathcal{N}(\mathbf{z}; \mu_{\boldsymbol{\phi}}(\mathbf{x}_n), \mathrm{diag}(\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}_n)))$.

Since we use a stochastic optimization algorithm anyway, we choose $K = 1$, i.e., we use only one Monte Carlo sample for each training data sample. After neglecting additive constant terms and scaling by $N/2$, we obtain the cost function
\begin{equation*}
 J(\boldsymbol{\theta}, \boldsymbol{\phi}) = 784 \log{\sigma_{\boldsymbol{\theta}}^2} + \frac{1}{N \sigma_{\boldsymbol{\theta}}^2} \sum_{n=1}^N  \|\mu_{\boldsymbol{\theta}}(\mathbf{z}_n) - \mathbf{x}_n\|^2_2 + \frac{1}{N} \sum_{n=1}^N \sum_{i=1}^2 \left( (\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}_n))_i + (\mu_{\boldsymbol{\phi}}(\mathbf{x}_n))^2_i - \log (\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}_n))_i\right),
\end{equation*}
where $\mathbf{z}_n$ are samples from $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x}) = \mathcal{N}(\mathbf{z}; \mu_{\boldsymbol{\phi}}(\mathbf{x}_n), \mathrm{diag}(\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}_n)))$. We will minimize this cost function with stochastic gradient descent.

### Reparameterization trick

However, there is one problem: it is not immediately clear how to differentiate through the sampling operation of $\mathbf{z}_n$ with respect to the parameters $\boldsymbol{\phi}$ that determine the mean and variance of the distribution we sample from.

The solution is: We can draw a sample $\mathbf{z}_n$ from $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x}) = \mathcal{N}(\mathbf{z}; \mu_{\boldsymbol{\phi}}(\mathbf{x}_n), \mathrm{diag}(\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}_n)))$ by sampling $\boldsymbol{\epsilon}_n$ from the normal distribution $\mathcal{N}(\boldsymbol{0}, \mathbf{I}_2)$ and setting $\mathbf{z}_n = \mu_{\boldsymbol{\phi}}(\mathbf{x}_n) + \mathrm{diag}(\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}_n)) \boldsymbol{\epsilon}_n$.

Thus we can rewrite the cost function as
\begin{equation*}
    \begin{split}
 J(\boldsymbol{\theta}, \boldsymbol{\phi}) &= 784 \log{\sigma_{\boldsymbol{\theta}}^2} + \frac{1}{N \sigma_{\boldsymbol{\theta}}^2} \sum_{n=1}^N  \|\mu_{\boldsymbol{\theta}}(\mu_{\boldsymbol{\phi}}(\mathbf{x}_n) + \mathrm{diag}(\sigma_{\boldsymbol{\phi}}(\mathbf{x}_n)) \boldsymbol{\epsilon}_n) - \mathbf{x}_n\|^2_2 \\
 &\quad + \frac{1}{N} \sum_{n=1}^N \sum_{i=1}^2 \left( (\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}_n))_i + (\mu_{\boldsymbol{\phi}}(\mathbf{x}_n))^2_i - \log (\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}_n))_i\right),
 \end{split}
\end{equation*}
where $\boldsymbol{\epsilon}_n$ are samples from the normal distribution $\mathcal{N}(\boldsymbol{0}, \mathbf{I}_2)$. Rewriting the cost function in this form is known as reparameterization trick and allows us to differentiate the cost function with respect to $\boldsymbol{\phi}$ in a straightforward way, since the samples $\boldsymbol{\epsilon}_n$ do not depend on the parameters anymore.

### Task

Complete the following implementation `cost_function(X, model)` of the cost function $J(\boldsymbol{\theta})$ that takes the data matrix $\mathbf{X}$ (each row corresponds to one image) and the nonlinear `model` as inputs. In the code, the computation of the term
\begin{equation}
\frac{1}{N} \sum_{n=1}^N \sum_{i=1}^2 \left( (\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}_n))_i + (\mu_{\boldsymbol{\phi}}(\mathbf{x}_n))^2_i - \log (\sigma_{\boldsymbol{\phi}}^2(\mathbf{x}_n))_i\right),
\end{equation}
that originates from the KL divergence expression in the ELBO, is missing.

*Hint*: You can make use of the PyTorch functions [`torch.mean`](https://pytorch.org/docs/stable/torch.html#torch.mean), [`torch.sum`](https://pytorch.org/docs/stable/torch.html#torch.sum), [`torch.exp`](https://pytorch.org/docs/stable/torch.html#torch.exp), and [`torch.pow`](https://pytorch.org/docs/stable/torch.html#torch.pow) in your implementation.

In [ ]:
def cost_function(X, model):
    # compute mean and log variance of the encoding distribution for input X
    Z_mu, Z_logsigma2 = model.encode(X)

    # compute the average KL divergence to the prior standard normal
    # distribution of z, neglecting additive constant terms
    # expected log p(x | z) + C
    # WRITE YOUR CODE HERE

    # sample encodings Z
    Z = Z_mu + torch.exp(0.5 * Z_logsigma2) * torch.randn_like(Z_logsigma2)

    # compute the (mean) decodings of Z
    X_decodings = model.decode(Z)

    # compute negative average log evidence of the input X, neglecting additive constant terms
    neg_log_evidence = 784 * model.logsigma2 + \
        torch.sum((X_decodings - X.view(-1, 784)).pow(2) * torch.exp(- model.logsigma2), dim=1).mean()

    return neg_log_evidence + kl


Now we can train the nonlinear model.

In [ ]:
# define the model and move it to the GPU if available
model = VAE()
model = model.to(device)

# define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

# track the training and test loss
training_loss = []
test_loss = []

# optimize parameters for 20 epochs
for i in range(20):
    # for each minibatch
    for batch_X, _ in train_data:
        # move batch to the GPU if available
        batch_X = batch_X.to(device)

        # reset the gradient information
        optimizer.zero_grad()

        # evaluate the cost function on the training data set
        loss = cost_function(batch_X, model)

        # update the statistics
        training_loss.append(loss.item())
        test_loss.append(float('nan'))

        # perform backpropagation
        loss.backward()

        # perform a gradient descent step
        optimizer.step()

    # evaluate the model after every epoch
    with torch.no_grad():
        # evaluate the cost function on the test data set
        accumulated_loss = 0
        for batch_X, _ in test_data:
            # move batch to the GPU if available
            batch_X = batch_X.to(device)

            # compute loss
            loss = cost_function(batch_X, model)
            accumulated_loss += loss.item()

        # update the statistics
        test_loss[-1] = accumulated_loss / len(test_data)
            
    print(f"Epoch {i + 1:2d}: training loss {training_loss[-1]: 9.3f}, "
          f"test loss {test_loss[-1]: 9.3f}")
        
# plot the tracked statistics
plt.figure()
iterations = np.arange(1, len(training_loss) + 1)
plt.scatter(iterations, training_loss, label='training loss')
plt.scatter(iterations, test_loss, label='test loss')
plt.legend()
plt.xlabel('iteration')
plt.show()

### Task

Read through and try to understand the implementation of the training procedure above. Answer the following questions:

- How does it differ from the implementation of the training procedure of the nonlinear model above?
- How were the parameters initialized?
- What learning rate did we use in the gradient descent algorithm?
- For how many iterations was the model trained?

## Encodings

In contrast to the regular nonlinear model without encoder, the VAE allows us to encode the images of the MNIST data set in the two-dimensional latent space.

Similar to probabilistic PCA, there exists not one unique representation of an image in the latent space but instead each image $\mathbf{x}$ gives rise to a whole distribution $q_{\boldsymbol{\phi}}(\mathbf{z}; \mathbf{x})$ of representations in the latent space. Here we use the mean $\mu_{\boldsymbol{\phi}}(\mathbf{x})$ of the encoding distribution as encoding. Alternatively, one could e.g. sample from the encoding distribution.

### Task

Compute the two-dimensional encodings $\mu_{\boldsymbol{\phi}}(\mathbf{x})$ of the images in the MNIST training and test data sets. Make use of the function `plot_encodings` below and visualize them with 2D scatter plots.

In [ ]:
# plot `train_encodings` and `test_encodings` with colorcoding of the corresponding labels
def plot_encodings(train_encodings, train_labels, test_encodings, test_labels):
    # create two plots side by side
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

    # plot encodings of training data
    ax = axes[0]
    ax.scatter(
        train_encodings[:, 0].cpu().numpy(), train_encodings[:, 1].cpu().numpy(),
        c=train_labels, cmap=plt.cm.tab10, vmin=-0.5, vmax=9.5, alpha=0.7
    )
    ax.set_xlabel("$z_1$")
    ax.set_ylabel("$z_2$")
    ax.set_title("training data")
    
    # plot encodings of test data
    ax = axes[1]
    scatter = ax.scatter(
        test_encodings[:, 0].cpu().numpy(), test_encodings[:, 1].cpu().numpy(),
        c=test_labels, cmap=plt.cm.tab10, vmin=-0.5, vmax=9.5, alpha=0.7
    )
    ax.set_xlabel("$z_1$")
    ax.set_ylabel("$z_2$")
    ax.set_title("test data")

    # add colorbar
    cb = fig.colorbar(scatter, ticks=range(10), ax=axes.ravel().tolist())
    cb.ax.set_title("digit")
    
    return fig

### Task

Compare the encodings of the test images with the encodings for regular PCA in exercise 8.1 and probabilistic PCA. Are they different and, if yes, in what ways?

## Decodings

Of course, we can also decode the representations in the latent space according to the definition of
the decoding distribution
\begin{equation*}
  p_{\boldsymbol{\theta}}(\mathbf{x} \,|\, \mathbf{z}) = \mathcal{N}\left(\mathbf{x}; \mu_{\boldsymbol{\theta}}(\mathbf{z}), \sigma_{\boldsymbol{\theta}}^2 \mathbf{I}_{784}\right).
\end{equation*}
Analogously to the encodings discussed above and the probabilistic PCA model, here we take the mean $\mu_{\boldsymbol{\theta}}(\mathbf{z})$ of the decoding distribution as decoding of $\mathbf{z}$. Alternatively we could e.g. sample from the decoding distribution.

### Task

Compute the reconstructions of the test images by mapping the encodings of the test data from the latent space back to the space of images.

### Task

As in the previous parts of the lab session, we compare the test images with their reconstructions to see how much information we lose by encoding the MNIST images in a two-dimensional space. Plot a set of images and their reconstructions with the function `plot_reconstructions` below.

In [ ]:
# plot a grid of random pairs of `originals` and `reconstructions`
def plot_reconstructions(originals, reconstructions, labels, num_rows=4, num_cols=2):
    # indices of displayed samples
    n = originals.shape[0]
    indices = np.random.choice(n, size=num_rows*num_cols, replace=False)

    fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(10, 10))
    for (idx, ax) in zip(indices, axes.flat):
        # extract original, reconstruction, and label
        original = originals[idx]
        reconstruction = reconstructions[idx]
        label = labels[idx]

        # configure subplot
        ax.set_xticks(())
        ax.set_yticks(())
        ax.grid(False)
        ax.set_title(f"Label: {label.item()}", fontweight='bold')

        # plot original and reconstructed image in a grid
        grid = np.ones((32, 62))
        grid[2:30, 2:30] = original.view(28, 28).cpu().numpy()
        grid[2:30, 32:60] = reconstruction.view(28, 28).cpu().numpy()
        ax.imshow(grid, vmin=0.0, vmax=1.0, cmap='gray_r')

    return fig

### Task

Compute the average squared reconstruction error
\begin{equation*}
    \mathrm{sqerr} := \frac{1}{10000} \sum_{i=1}^{10000} \|\mathbf{x}_i - \tilde{\mathbf{x}}_i\|^2_2
\end{equation*}
of the images $\mathbf{x}_i \in {[0,1]}^{784}$ and their reconstructions $\tilde{\mathbf{x}}_i \in \mathbb{R}^{784}$ ($i = 1,\ldots, 10000$) in the MNIST test data set as an objective measure for the quality of the reconstructions.

### Task

Now we have performed exactly the same analysis as for the regular PCA in exercise 8.1 and the probabilistic PCA. Compare the results you obtained with regular PCA, probabilistic PCA, and the VAE, and answer the following questions:
- Which digits can be reconstructed and decoded quite well, and which ones seem to be more challenging? Are there differences between the three approaches?
- Is the average squared reconstruction error the same as with the other two approaches?

## Generating images

The VAE allows to generate new MNIST-like images in the same way as for the nonlinear model without encoder.

### Task

#### (a)

Generate 25 images by sampling encodings from $\mathcal{N}(\boldsymbol{0}, \mathbf{I}_2)$ and decoding them. Plot them with the function `plot_images`. Are the samples visually better than the samples from the nonlinear model without encoder or the samples from the probabilistic PCA model?

#### (b)

Perform sampling with different temperatures and study the quality of the samples.

## Summary

We have trained a variational autoencoder that allows us to generate MNIST-like images. Adding a nonlinearity and an encoder seems to improve the quality of the samples. However, we also notice that the model is not perfect. Many further modifications of the decoder and encoder models are possible and could potentially improve the sampler. For instance, the dimension of the latent space can be increased (then the information loss by encoding the images in the latent space should be reduced). Alternatively the nonlinear decoder model can be changed: a more flexible model with increased number of layers in the neural network or a diagonal (or even full) covariance matrix could be used, or the outputs could be restricted to values between 0 and 1 (since we represent MNIST images as vectors with entries between 0 and 1).